disciplina-turma-nota
Perguntas:<br>
* Notas das turmas tem muita diferença entre um mesmo semestre?

In [66]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from bokeh.plotting import figure, output_notebook, show
from bokeh.layouts import gridplot, row
from bokeh.models.widgets import Tabs, Panel
%matplotlib inline

In [70]:
pd_dados = pd.read_csv('turma_professor_notas.csv')
pd_disciplinas = pd.read_csv('disciplinas.csv')

In [86]:
listTabs = []
for index, row in pd_disciplinas.iterrows():
    
    y = pd_dados[pd_dados['nome_disc'] == row['nome_disc']]['reprovados']
    x = []
    for index_disc, row_disc in pd_dados[pd_dados['nome_disc'] == row['nome_disc']].iterrows():
        if (int(row_disc['periodo']) == 1):
            x.append(str(row_disc['ano'])+""+str(0))
        else:
           x.append(str(row_disc['ano'])+""+str(5))
        
    p = figure(plot_width=400, plot_height=300, title=row['nome_disc'], x_axis_label='ano/semestre', y_axis_label='Quant Reprovados')
    p.circle(x, y, fill_color="white", size=12)
    listTabs.append(p)

output_notebook()
grid = gridplot([[ listTabs[0], listTabs[1]],[ listTabs[2], listTabs[3]]])
show(grid)

['20140', '20140', '20140', '20140', '20140', '20140', '20140', '20140', '20145', '20145', '20145', '20145', '20150', '20150', '20150', '20150', '20150', '20150', '20150', '20150', '20155', '20155', '20155', '20155', '20155', '20155', '20160', '20160', '20160', '20160', '20160', '20160', '20160', '20160', '20160', '20160', '20165', '20165', '20165', '20165', '20170', '20170', '20170', '20170', '20170', '20170', '20170', '20170', '20170', '20170']
['20140', '20140', '20145', '20145', '20145', '20150', '20150', '20150', '20155', '20155', '20155', '20155', '20155', '20160', '20160', '20160', '20160', '20165', '20165', '20165', '20165', '20170', '20170', '20170']
['20140', '20140', '20145', '20145', '20145', '20150', '20150', '20150', '20150', '20155', '20155', '20155', '20155', '20160', '20160', '20160', '20165', '20165', '20165', '20165', '20165', '20170', '20170', '20170', '20170', '20170']
['20140', '20145', '20145', '20150', '20150', '20155', '20155', '20160', '20160', '20160', '20165

Loading BokehJS ...

In [92]:
listTabs = []
for index, row in pd_disciplinas.iterrows():
    x = ['20140', '20145', '20150', '20155', '20160', '20165', '20170']

    listCount = []
    for count in pd_dados.groupby(['nome_disc', 'ano_periodo'])['total_alunos'].sum()[row['nome_disc']]:
        listCount.append(count)

    p = figure(plot_width=400, plot_height=300, title=row['nome_disc'], x_axis_label='ano/semestre', y_axis_label='Total Matriculados')
    p.line(x, listCount)
    listTabs.append(p)

output_notebook()
grid = gridplot([[ listTabs[0], listTabs[1]],[ listTabs[2], listTabs[3]]])
show(grid)

Loading BokehJS ...

In [106]:
listTabsX = []
listTabsY = []
for index, row in pd_disciplinas.iterrows():
    x = ['20140', '20145', '20150', '20155', '20160', '20165', '20170']
    listTabsX.append(x)
    listCount = []
    for count in pd_dados.groupby(['nome_disc', 'ano_periodo'])['total_alunos'].sum()[row['nome_disc']]:
        listCount.append(count)

    
    listTabsY.append(listCount)
    #listTabs.append(p)

output_notebook()
p = figure(plot_width=800, plot_height=600, title='Disciplinas', x_axis_label='ano/semestre', y_axis_label='Total Matriculados')
p.multi_line(legend="teste", xs=[listTabsX[0], listTabsX[1], listTabsX[2], listTabsX[3]], ys=[listTabsY[0],listTabsY[1],listTabsY[2],listTabsY[3]], line_width=4)
show(p)

Loading BokehJS ...